# Cluster Centroid (undersampling) SMOTE (oversampling) SMOTEEN (combo)
# workflow
### Import the data.
### Separate the data into target and feature datasets.
### Encode the data.
### Split the data into training and testing datasets.
### Scale the data.
### Import the resampling technique (random oversampling, cluster centroids, SMOTE, etc.)
### Resample the X and y training sets to make the value counts between classes approximately equal.
### Instantiate the classifier model.
### Fit the model with the resampled data
### Generate the predictions.
### Review the results.

In [1]:
# Import modules
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

In [2]:
# Read the data from the CSV file into a Pandas DataFrame
df = pd.read_csv(
    Path('bank.csv')
)

# Review the DataFrame
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [3]:
# Seperate the features data, X, from the target data, y
y = df['y']
X = df.drop(columns='y')

In [4]:
# Encode the features dataset's categorical variables using get_dummies
X = pd.get_dummies(X)

# Review the features DataFrame
X.head()

,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,30,1787,19,79,1,-1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,33,4789,11,220,1,339,4,0,0,0,...,0,0,1,0,0,0,1,0,0,0
2,35,1350,16,185,1,330,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,30,1476,3,199,4,-1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
4,59,0,5,226,1,-1,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1


In [5]:
# Split data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [6]:
# Review the distinct values from y
y_train.value_counts()

no     3012
yes     378
Name: y, dtype: int64

In [7]:
# Instantiate a StandardScaler instance
scaler = StandardScaler()

# Fit the training data to the standard scaler
X_scaler = scaler.fit(X_train)

# Transform the training data using the scaler
X_train_scaled = X_scaler.transform(X_train)

# Transform the testing data using the scaler
X_test_scaled = X_scaler.transform(X_test)

In [8]:
# Import the RandomForestClassifier from sklearn
from sklearn.ensemble import RandomForestClassifier

# Instantiate a RandomForestClassifier instance
model = RandomForestClassifier()

# Fit the traning data to the model
model.fit(X_train_scaled, y_train)

RandomForestClassifier()

In [9]:
# Predict labels for original scaled testing features
y_pred = model.predict(X_test_scaled)

# Cluster Centroids

In [10]:
# Import ClusterCentroids from imblearn
from imblearn.under_sampling import ClusterCentroids

# Instantiate a ClusterCentroids instance
cc_sampler = ClusterCentroids(random_state=1)

In [11]:
# Fit the training data to the cluster centroids model
X_resampled, y_resampled = cc_sampler.fit_resample(X_train_scaled, y_train)

In [12]:
# Count distinct values for the resampled target data
y_resampled.value_counts()

no     378
yes    378
Name: y, dtype: int64

In [13]:
# Instantiate a new RandomForestClassier model
cc_model = RandomForestClassifier()

# Fit the resampled data the new model
cc_model.fit(X_resampled, y_resampled)

RandomForestClassifier()

In [14]:
# Predict labels for resampled testing features
cc_y_pred = cc_model.predict(X_test_scaled)

In [15]:
# Print classification reports
print(f"Classifiction Report - Original Data")
print(classification_report(y_test, y_pred))
print("---------")
print(f"Classifiction Report - Redsampled Data - CentroidClusters")
print(classification_report(y_test, cc_y_pred))

Classifiction Report - Original Data
              precision    recall  f1-score   support

          no       0.89      0.98      0.94       988
         yes       0.60      0.19      0.29       143

    accuracy                           0.88      1131
   macro avg       0.75      0.59      0.61      1131
weighted avg       0.86      0.88      0.85      1131

---------
Classifiction Report - Redsampled Data - CentroidClusters
              precision    recall  f1-score   support

          no       0.99      0.33      0.49       988
         yes       0.18      0.99      0.30       143

    accuracy                           0.41      1131
   macro avg       0.58      0.66      0.40      1131
weighted avg       0.89      0.41      0.47      1131



# SMOTE

In [17]:
# Import SMOTE from imblearn
from imblearn.over_sampling import SMOTE

# Instantiate the SMOTE instance 
# Set the sampling_strategy parameter equal to auto
smote_sampler = SMOTE(random_state=1, sampling_strategy='auto')

In [18]:
# Fit the training data to the smote_sampler model
X_resampled, y_resampled = smote_sampler.fit_resample(X_train_scaled, y_train)

In [19]:
# Count distinct values for the resampled target data
y_resampled.value_counts()

no     3012
yes    3012
Name: y, dtype: int64

In [20]:
# Instantiate a new RandomForestClassier model 
smote_model = RandomForestClassifier()

# Fit the resampled data to the new model
smote_model.fit(X_resampled, y_resampled)

RandomForestClassifier()

In [21]:
# Predict labels for resampled testing features
smote_y_pred = smote_model.predict(X_test_scaled)

In [22]:
# Print classification reports
print(f"Classifiction Report - Original Data")
print(classification_report(y_test, y_pred))
print("---------")
print(f"Classifiction Report - Redsampled Data - SMOTE")
print(classification_report(y_test, smote_y_pred))

Classifiction Report - Original Data
              precision    recall  f1-score   support

          no       0.89      0.98      0.94       988
         yes       0.60      0.19      0.29       143

    accuracy                           0.88      1131
   macro avg       0.75      0.59      0.61      1131
weighted avg       0.86      0.88      0.85      1131

---------
Classifiction Report - Redsampled Data - SMOTE
              precision    recall  f1-score   support

          no       0.90      0.96      0.93       988
         yes       0.53      0.28      0.37       143

    accuracy                           0.88      1131
   macro avg       0.72      0.62      0.65      1131
weighted avg       0.86      0.88      0.86      1131



# SMOTEENN

In [24]:
# Import SMOTEEN from imblearn
from imblearn.combine import SMOTEENN

# Instantiate the SMOTEENN instance
smote_enn = SMOTEENN(random_state=1)

# Fit the model to the training data
X_resampled, y_resampled = smote_enn.fit_resample(X_train_scaled, y_train)

# Count distinct values for the resampled target data
y_resampled.value_counts()

# Instantiate a new RandomForestClassier model
smoteenn_model = RandomForestClassifier()

# Fit the resampled data the new model
smoteenn_model.fit(X_resampled, y_resampled)

# Predict labels for resampled testing features
smoteenn_y_pred = smoteenn_model.predict(X_test_scaled)

# Print classification reports
print(f"Classifiction Report - Original Data")
print(classification_report(y_test, y_pred))
print("---------")
print(f"Classifiction Report - Redsampled Data - SMOTEENN")
print(classification_report(y_test, smoteenn_y_pred))

Classifiction Report - Original Data
              precision    recall  f1-score   support

          no       0.89      0.98      0.94       988
         yes       0.60      0.19      0.29       143

    accuracy                           0.88      1131
   macro avg       0.75      0.59      0.61      1131
weighted avg       0.86      0.88      0.85      1131

---------
Classifiction Report - Redsampled Data - SMOTEENN
              precision    recall  f1-score   support

          no       0.93      0.91      0.92       988
         yes       0.45      0.49      0.47       143

    accuracy                           0.86      1131
   macro avg       0.69      0.70      0.69      1131
weighted avg       0.86      0.86      0.86      1131

